# Część 1

Rozwiązywany problem: Mój Model ma za
zadanie zdecydować czy grzyb jest jadalny czy niejadalny.
Na podstawie Datasetu (Mushroom Dataset),




**O kolumnach**
*   cap-diameter: średnica kapelusza,
*    cap-shape: kształt kapelusza,
*    gill-attachment: przyrośnięcie blaszek,
*    gill-color: kolor blaszek,
*    stem-height: wysokość trzonu,
*    stem-width: szerokość trzonu,
*    stem-color: kolor trzonu,
*    season: pora roku,
*    class: klasa (jadalny, trujący).






> "Wszystkie grzyby są jadalne...
                        ale niektóre tylko raz".





In [ ]:
#Wczytanie bibliotek
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

**Wczytanie/Oczyszczanie Danych, oraz głębszy wgląd do Datasetu**


In [ ]:
dataset_path = '/content/mushroom.csv'

#Wczytanie Dataset
df = pd.read_csv(dataset_path)

In [ ]:
df = df.drop_duplicates() #Usuwamy dublikaty wierszy
df.dropna(inplace=True) #Usuwamy puste wartości

df.info() #  Wyświetlamy podstawowe informacjie o naszym DataFrame

<class 'pandas.core.frame.DataFrame'>
Index: 53732 entries, 0 to 54034
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cap-diameter     53732 non-null  int64  
 1   cap-shape        53732 non-null  int64  
 2   gill-attachment  53732 non-null  int64  
 3   gill-color       53732 non-null  int64  
 4   stem-height      53732 non-null  float64
 5   stem-width       53732 non-null  int64  
 6   stem-color       53732 non-null  int64  
 7   season           53732 non-null  float64
 8   class            53732 non-null  int64  
dtypes: float64(2), int64(7)
memory usage: 4.1 MB


In [ ]:
df

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
0,1372,2,2,10,3.807467,1545,11,1.804273,1
1,1461,2,2,10,3.807467,1557,11,1.804273,1
2,1371,2,2,10,3.612496,1566,11,1.804273,1
3,1261,6,2,10,3.787572,1566,11,1.804273,1
4,1305,6,2,10,3.711971,1464,11,0.943195,1
...,...,...,...,...,...,...,...,...,...
54030,73,5,3,2,0.887740,569,12,0.943195,1
54031,82,2,3,2,1.186164,490,12,0.943195,1
54032,82,5,3,2,0.915593,584,12,0.888450,1
54033,79,2,3,2,1.034963,491,12,0.888450,1


from matplotlib import pyplot as plt
df['cap-diameter'].plot(kind='hist', bins=20, title='cap-diameter')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['cap-shape'].plot(kind='hist', bins=20, title='cap-shape')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['gill-attachment'].plot(kind='hist', bins=20, title='gill-attachment')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['gill-color'].plot(kind='hist', bins=20, title='gill-color')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='gill-color', y='stem-height', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['cap-diameter'].plot(kind='line', figsize=(8, 4), title='cap-diameter')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['cap-shape'].plot(kind='line', figsize=(8, 4), title='cap-shape')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['gill-attachment'].plot(kind='line', figsize=(8, 4), title='gill-attachment')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['gill-color'].plot(kind='line', figsize=(8, 4), title='gill-color')
plt.gca().spines[['top', 'right']].set_visible(False)

**W kolumnach nie ma brakującyhc danych, oraz nasz Dataset nie ma żadnych danych Stringowych, posiada tylko dane liczbowe**

Niżej sprawdzamy po kolumnie "class", ile jest grzybów jadalnych(0), lub trujących(1)

In [ ]:
counts = df['class'].value_counts(normalize=True) * 100

counts

class
1    54.663887
0    45.336113
Name: proportion, dtype: float64

Dzielimy dane na treningowe oraz testowe , stosujemy podział prosty, ponieważ mamy małą dysproporcję między grzybami 1(Trującymi) a 0(Jadalnymi)


In [ ]:
X = df.drop(columns=['class'])
Y = df['class']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,random_state=42,stratify=Y)

**Skalowanie i Transformacja**

Używamy MinMaxScaler ponieważ nasz DF posiada duże wachania między wynikami.

In [ ]:
scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [ ]:
X_train_scaled.shape[0] #Dane treningowe

42985

In [ ]:
X_test_scaled.shape[0] #Dane testowe

10747

# Część 2

Wybór algorytmu modelu, oraz trenowanie modelu.

Zdecydowałem się wybrac model Random Forest, ze względu na jego dokładność, dobrą zdolność do generalizacji(co oznacza, że dobrze radzi sobie z klasyfikacją nieznanych wcześniej przykładów), oraz ze względu na jego odporność na przetrenowanie.

In [ ]:
#Tworzenie potoku
pip = Pipeline([
    ('scaler', MinMaxScaler()),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [ ]:
#Definiujemy przestrzeń hiperparametrów
param_grid = {
    'classifier__n_estimators': [50, 150, 200],
    'classifier__max_depth': [5, 10, 15],
    'classifier__min_samples_split': [2, 4, 8]
}

In [ ]:
grid_search = GridSearchCV(pip, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train_scaled, Y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'classifier__max_depth': [5, 10, 15],
                         'classifier__min_samples_split': [2, 4, 8],
                         'classifier__n_estimators': [50, 150, 200]},
             scoring='f1')

In [ ]:
paramas = grid_search.best_params_
paramas #Najlepsze Hiperparametry

{'classifier__max_depth': 15,
 'classifier__min_samples_split': 2,
 'classifier__n_estimators': 200}

In [ ]:
Y_pred = grid_search.predict(X_test_scaled) #Predykcja
Y_pred_train = grid_search.predict(X_train_scaled)

**Ocena naszego modelu**

In [ ]:
print(classification_report(Y_train, Y_pred_train))
print(f'Accuracy: {accuracy_score(Y_train, Y_pred_train)}')

              precision    recall  f1-score   support

           0       0.99      1.00      0.99     19488
           1       1.00      0.99      0.99     23497

    accuracy                           0.99     42985
   macro avg       0.99      0.99      0.99     42985
weighted avg       0.99      0.99      0.99     42985

Accuracy: 0.9934163080144236


In [ ]:
print(classification_report(Y_test, Y_pred))  # Porównanie
print(f'Accuracy: {accuracy_score(Y_test, Y_pred)}')

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      4872
           1       0.99      0.99      0.99      5875

    accuracy                           0.99     10747
   macro avg       0.99      0.99      0.99     10747
weighted avg       0.99      0.99      0.99     10747

Accuracy: 0.9865078626593468


Jak widać nasz model ma wysoką precyzje (Określa jak wiele z przewidywanych jako pozytywne przypadki są faktycznie pozytywne), wysoki Recall (Określa jak wiele z rzeczywiście pozytywnych przypadków zostało poprawnie wykrytych przez model) oraz wysoki Accuracy(Określa ogólną skuteczność modelu).

**Macro avg** to średnia arytmetyczna dla każdej metryki obliczona niezależnie dla każdej klasy. Traktuje wszystkie klasy jednakowo, niezależnie od ich liczebności.

**Weighted avg** to średnia ważona dla każdej metryki, uwzględniająca liczbę przykładów w każdej klasie. Traktuje każdą klasę proporcjonalnie do jej liczebności w zbiorze danych.




In [ ]:
print("Matryca pomyłek dla zbioru testowego:")
print(confusion_matrix(Y_test, Y_pred))

Matryca pomyłek dla zbioru testowego:
[[4811   61]
 [  84 5791]]


Matryca pomyłek:


True Positives (TP): 5791

Model poprawnie sklasyfikował 5791 przykładów jako pozytywne (np. jadalne grzyby).
False Negatives (FN): 84

Model błędnie sklasyfikował 84 przykładów jako negatywne (np. uznał jadalne grzyby za trujące).
True Negatives (TN): 4811

Model poprawnie sklasyfikował 4811 przykładów jako negatywne (np. trujące grzyby).
False Positives (FP): 61

Model błędnie sklasyfikował 61 przykładów jako pozytywne (np. uznał trujące grzyby za jadalne).

**Dodatkowo walidacja krzyżowa (Cross-Validation)**



In [ ]:
best_model = grid_search.best_estimator_
scores = cross_val_score(best_model, X_train_scaled, Y_train, cv=5, scoring='accuracy')
print("Cross-validation scores:", scores)
print("Average cross-validation score:", np.mean(scores))

Cross-validation scores: [0.98894963 0.98569268 0.98836804 0.9881354  0.98685588]
Average cross-validation score: 0.9876003256950099


Wyniki te sugerują, że model zachowuje się dobrze na różnych podziałach danych treningowych i walidacyjnych, co sugeruje, że model nie jest zbyt przetrenowany na danych treningowych i generalizuje dobrze na nowe dane